<img src="../intro-slide-wids22.svg"
     max_width=100%
     height=auto
     alt="WiDS title slide" />

<center>
  <img src="https://raw.githubusercontent.com/dask/marketing/main/source/images/dask-horizontal.svg" alt="Dask logo" width=55%/>
</center>

# Materials and setup

Tutorial materials available at https://github.com/coiled/micro-dask-tutorial-widsPS22. Two ways to go through the tutorial:
- Run using Binder (no setup required, recommended for today!)
- Run locally on your laptop (recommended for going back through materials later)

# A common problem

Timseries dataset for one week, with one row per second (publicly accessible, see [this github repo](https://github.com/coiled/coiled-datasets) for more background). Using pandas, it's very easy to take one week of this dataset and calculate the counts of unique values for a given column:

```python
import pandas as pd
df = pd.read_parquet("s3://coiled-datasets/timeseries/20-years/parquet/part.0.parquet")
df['name'].value_counts().astype(int)
```

But what if I want to look at more than 1 week? For 20 year's worth, the compressed size is 16.7 GB and the uncompressed size is 58.2 GB

```
20-years
└── parquet
    ├── part.0.parquet
    ├── part.1.parquet
    ├── part.2.parquet
   ...
    ├── part.1094.parquet
```

One solution, is to read and process this dataset into chunks:

```python
files = pathlib.Path("../data/timeseries/20-years/parquet/").glob("part.*.parquet")
counts = pd.Series(dtype=int)
for path in files:
    # read in each chunk
    df = pd.read_parquet(path)
    # sum counts as you go, storing each chunk
    counts = counts.add(df['name'].value_counts(), fill_value=0)
    counts.astype(int)
```

But each of these operations doesn't need to happen serially, we could read the chunks and calculate the number of unique values in parallel, and sum at the end. The tricky part is figuring out how to break the computation into chunks, which in this example was fairly straightforward, but other computations can become more complicted, which is where Dask comes in!

# What is Dask?

Dask is an open-source parallel & distributed computing framework for Python, which supports __larger-than-memory computation__, enabling data processing and modeling for datasets that don’t fit in RAM. Dask is used in a wide range of domains from finance and retail to academia and life sciences (check out [this video](https://youtu.be/t_GRK4L-bnw) or [blog post](https://coiled.io/blog/who-uses-dask/) on who uses Dask). It is also leveraged internally by numerous special-purpose tools like XGBoost, RAPIDS, PyTorch, Prefect, and Airflow. It’s developed in ongoing collaboration with the PyData community making it easy to learn, integrate, and operate. The familiarity of collections like Dask DataFrame to pandas allow you to quickly get started on your hardware of choice– be it your laptop, cloud service, or HPC cluster.

# How can I use Dask?

If you are familiar with Numpy, pandas and scikit-learn then think of Dask as their faster cousin. Take this example comparing the syntax for a simple groupby using pandas versus Dask DataFrame:

```python
# pandas syntax                       # dask dataframe syntax
import pandas as pd                   import dask.dataframe as dd
df = pd.read_csv('2015-01-01.csv')    df = dd.read_csv('2015-*-*.csv')
df.groupby('user_id').value.mean()    df.groupby('user_id').value.mean().compute()
```

# How does Dask work?

We can think of Dask at a high- and low- level. The high-level colletions Array, Bag, and DataFrame mimic NumPy, lists, and pandas but can operate in parallel on datasets that don’t fit into memory. Delayed and Futures are low-level collections for custom computations. In addition to these high-level collections, Dask provides dynamic task schedulers that execute the task graphs created from high-level collections and custom workloads.

<figure>
  <img src="https://raw.githubusercontent.com/dask/marketing/main/source/images/dask-overview.svg" 
     width="70%"
     alt="Dask overview">
  <figcaption>In this workshop, we'll be focusing on the Dask DataFrame collection.</figcaption>
</figure>

# When should I use Dask?

Do your data fit comfortably in memory and your computations are already blazingly fast? Love that for you! This means Dask *probably* isn't the right tool, though. You'll see the biggest performance improvements for problems that are memory-bound or compute-bound (or both). If you don't have any of these problems, you're likely better off using pandas or scikit-learn directly. If you're still not sure, check out [Why Dask?](https://docs.dask.org/en/latest/why.html)


<img src="https://raw.githubusercontent.com/dask/dask-ml/main/docs/source/images/dimensions_of_scale.svg"
     width="40%"
     alt="Conceptual graph of size of the model vs. the size of the data."/>

__Bottom Left:__ You don't need Dask.
__Elsewhere:__ Dask is fair game.

# What we'll cover in the next 40 minutes

- Load and process Seattle bicycle counter data using the Dask Delayed, Bag, and Dask DataFrame collections
- (Time permitting) Forecast what the future of cycling might look like with Prophet
- Monitor these computations using the interactive dashboard
- Q&A